In [14]:
import torch
import torchvision

In [2]:
import arrayfire as af
import array

In [15]:
def toArrayFire(x):
    x_np = x.detach().contiguous().numpy()
    shape = 1
    if len(x_np.shape) == 0:
        shape = (1,)
    else:
        shape = x_np.shape[::-1]
    print(shape)
    afArray = af.Array(x_np.ctypes.data, shape, x_np.dtype.char)
    return afArray

def saveStateDict(module, filepath):
    i = 0
    for (name, param) in module.named_parameters():
        #param = module.state_dict()[name]
        print(name, "\t", param.size())
        if 'in_proj' in name:
            print(param.shape)
            q, k, v = param.chunk(3, dim=0)
            print('in_proj!')
            af.array.save_array(name + 'q', toArrayFire(q), filepath, True)
            af.array.save_array(name + 'k', toArrayFire(k), filepath, True)
            af.array.save_array(name + 'v', toArrayFire(k), filepath, True)
            continue
        if len(param.size()) > 0:
            af_array = toArrayFire(param)
            if 'fc' in name and 'weight' in name:
                af_array = af.array.transpose(af_array)
            af.array.save_array(name, af_array, filepath, True)
            i = i + 1
    print(i)
    for name in module.state_dict():
        if 'running' in name:
            print(name)
            af_array = toArrayFire(module.state_dict()[name])
            af.array.save_array(name, af_array, filepath + 'running', True)

In [16]:
from models.backbone import *
from models.position_encoding import *
from models.matcher import *

In [17]:
from models.transformer import *

In [27]:
batch_size = 1
embedding_size = 12
src_len = 5
src = torch.rand(src_len, batch_size, embedding_size)
model = TransformerEncoderLayer(embedding_size, 1, dropout=0.0, dim_feedforward=128)
output = model.forward(src)

after self attention tensor([[[ 0.2310, -0.4288, -0.2431,  0.0449, -0.4100,  0.3400,  0.1781,
          -0.1560, -0.1527, -0.3016,  0.5055, -0.0278]],

        [[ 0.2334, -0.4242, -0.2405,  0.0427, -0.4118,  0.3394,  0.1841,
          -0.1544, -0.1586, -0.2983,  0.5045, -0.0276]],

        [[ 0.2335, -0.4236, -0.2393,  0.0429, -0.4118,  0.3396,  0.1844,
          -0.1557, -0.1579, -0.2985,  0.5043, -0.0287]],

        [[ 0.2320, -0.4283, -0.2396,  0.0444, -0.4097,  0.3399,  0.1782,
          -0.1590, -0.1501, -0.3019,  0.5041, -0.0304]],

        [[ 0.2318, -0.4274, -0.2427,  0.0440, -0.4108,  0.3397,  0.1805,
          -0.1547, -0.1557, -0.3002,  0.5055, -0.0271]]],
       grad_fn=<AddBackward0>)
after dropout tensor([[[ 0.9534,  0.1263, -0.1960,  0.8470, -0.3142,  1.2343,  0.2509,
           0.1605,  0.7483,  0.0972,  0.7158,  0.4215]],

        [[ 1.1922,  0.3208, -0.1261,  0.3631,  0.2563,  0.5813,  1.1257,
           0.2529,  0.2898, -0.1682,  1.1010, -0.0240]],

        [[ 0.9180

In [28]:
filepath = '/private/home/padentomasello/scratch/pytorch_testing/transformer_encoder_layer.array'
af.array.save_array('input', toArrayFire(src), filepath, False)
i = 1
params = {}
for (name, param) in model.named_parameters():
        if 'in_proj' in name:
            q, k, v = param.chunk(3, dim=0)
            hack = '0'
            if 'in_proj_bias' in name: hack = '1'
            params['0q_' + hack + name] = q
            params['1k_' + hack + name] = k
            params['2v_' + hack + name] = v
            if 'in_proj_bias' in name:
                for key in sorted(params.keys()):
                    af_array = toArrayFire(params[key])
                    print(name)
                    if 'weight' in key:
                        print('here!')
                        af_array = af.array.transpose(af_array)
                    print(key, i)
                    af.array.save_array(key, af_array, filepath, True)  
                    i = i + 1
            continue
        elif len(param.size()) > 0:
            af_array = toArrayFire(param)
            if 'fc' in name and 'weight' in name:
                af_array = af.array.transpose(af_array)
            if 'weight' in name and 'proj' in name:
                af_array = af.array.transpose(af_array)
            if 'weight' in name and 'linear' in name:
                af_array = af.array.transpose(af_array)
            print(name, i)
            af.array.save_array(name, af_array, filepath, True)
            i = i + 1
#af.array.save_array('output', toArrayFire(output), filepath, True)
af.array.save_array('output', toArrayFire(output), filepath, True)

(12, 1, 5)
(12, 12)
self_attn.in_proj_bias
here!
0q_0self_attn.in_proj_weight 1
(12,)
self_attn.in_proj_bias
0q_1self_attn.in_proj_bias 2
(12, 12)
self_attn.in_proj_bias
here!
1k_0self_attn.in_proj_weight 3
(12,)
self_attn.in_proj_bias
1k_1self_attn.in_proj_bias 4
(12, 12)
self_attn.in_proj_bias
here!
2v_0self_attn.in_proj_weight 5
(12,)
self_attn.in_proj_bias
2v_1self_attn.in_proj_bias 6
(12, 12)
self_attn.out_proj.weight 7
(12,)
self_attn.out_proj.bias 8
(12, 128)
linear1.weight 9
(128,)
linear1.bias 10
(128, 12)
linear2.weight 11
(12,)
linear2.bias 12
(12,)
norm1.weight 13
(12,)
norm1.bias 14
(12,)
norm2.weight 15
(12,)
norm2.bias 16
(12, 1, 5)


17

In [20]:
output

tensor([[[ 0.8188, -0.8131, -1.1027, -1.2139,  1.3754,  0.2462, -1.2540,
           1.6625, -0.7953, -0.3104,  0.7027,  0.6837]],

        [[ 0.5006, -0.3333,  0.3904, -0.2153, -1.5290,  1.3152, -0.4167,
           1.3011, -1.0204, -1.7167,  0.7378,  0.9862]],

        [[ 0.6038, -0.6803, -0.2174, -0.1719,  1.1383, -0.7165, -1.2133,
           2.1317,  0.8927, -0.0180, -0.1876, -1.5614]],

        [[ 0.7928, -0.7490,  1.2031, -0.8406, -1.0975,  0.6829,  0.4576,
          -0.2225, -0.0312, -1.4745,  2.0058, -0.7268]],

        [[ 0.3869, -0.2117, -1.1351, -1.3118,  0.9864,  1.0178, -1.0721,
           1.3694, -1.2195,  1.2316, -0.6364,  0.5946]]],
       grad_fn=<NativeLayerNormBackward>)

In [21]:
import warnings
from typing import Tuple, Optional

import torch
from torch import Tensor
from torch.nn.modules.linear import _LinearWithBias
from torch.nn.init import xavier_uniform_
from torch.nn.init import constant_
from torch.nn.init import xavier_normal_
from torch.nn.parameter import Parameter
#from .module import Module
#from .. import functional as F

class MultiheadAttention2(nn.Module):
    r"""Allows the model to jointly attend to information
    from different representation subspaces.
    See reference: Attention Is All You Need

    .. math::
        \text{MultiHead}(Q, K, V) = \text{Concat}(head_1,\dots,head_h)W^O
        \text{where} head_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V)

    Args:
        embed_dim: total dimension of the model.
        num_heads: parallel attention heads.
        dropout: a Dropout layer on attn_output_weights. Default: 0.0.
        bias: add bias as module parameter. Default: True.
        add_bias_kv: add bias to the key and value sequences at dim=0.
        add_zero_attn: add a new batch of zeros to the key and
                       value sequences at dim=1.
        kdim: total number of features in key. Default: None.
        vdim: total number of features in value. Default: None.

        Note: if kdim and vdim are None, they will be set to embed_dim such that
        query, key, and value have the same number of features.

    Examples::

        >>> multihead_attn = nn.MultiheadAttention(embed_dim, num_heads)
        >>> attn_output, attn_output_weights = multihead_attn(query, key, value)
    """
    bias_k: Optional[torch.Tensor]
    bias_v: Optional[torch.Tensor]

    def __init__(self, embed_dim, num_heads, dropout=0., bias=True, add_bias_kv=False, add_zero_attn=False, kdim=None, vdim=None):
        super(MultiheadAttention2, self).__init__()
        self.embed_dim = embed_dim
        self.kdim = kdim if kdim is not None else embed_dim
        self.vdim = vdim if vdim is not None else embed_dim
        self._qkv_same_embed_dim = self.kdim == embed_dim and self.vdim == embed_dim

        self.num_heads = num_heads
        self.dropout = dropout
        self.head_dim = embed_dim // num_heads
        assert self.head_dim * num_heads == self.embed_dim, "embed_dim must be divisible by num_heads"

        if self._qkv_same_embed_dim is False:
            self.q_proj_weight = Parameter(torch.Tensor(embed_dim, embed_dim))
            self.k_proj_weight = Parameter(torch.Tensor(embed_dim, self.kdim))
            self.v_proj_weight = Parameter(torch.Tensor(embed_dim, self.vdim))
            self.register_parameter('in_proj_weight', None)
        else:
            self.in_proj_weight = Parameter(torch.empty(3 * embed_dim, embed_dim))
            self.register_parameter('q_proj_weight', None)
            self.register_parameter('k_proj_weight', None)
            self.register_parameter('v_proj_weight', None)

        if bias:
            self.in_proj_bias = Parameter(torch.empty(3 * embed_dim))
        else:
            self.register_parameter('in_proj_bias', None)
        self.out_proj = _LinearWithBias(embed_dim, embed_dim)

        if add_bias_kv:
            self.bias_k = Parameter(torch.empty(1, 1, embed_dim))
            self.bias_v = Parameter(torch.empty(1, 1, embed_dim))
        else:
            self.bias_k = self.bias_v = None

        self.add_zero_attn = add_zero_attn

        self._reset_parameters()

    def _reset_parameters(self):
        if self._qkv_same_embed_dim:
            xavier_uniform_(self.in_proj_weight)
        else:
            xavier_uniform_(self.q_proj_weight)
            xavier_uniform_(self.k_proj_weight)
            xavier_uniform_(self.v_proj_weight)

        if self.in_proj_bias is not None:
            constant_(self.in_proj_bias, 0.)
            constant_(self.out_proj.bias, 0.)
        if self.bias_k is not None:
            xavier_normal_(self.bias_k)
        if self.bias_v is not None:
            xavier_normal_(self.bias_v)

    def __setstate__(self, state):
        # Support loading old MultiheadAttention checkpoints generated by v1.1.0
        if '_qkv_same_embed_dim' not in state:
            state['_qkv_same_embed_dim'] = True

        super(MultiheadAttention, self).__setstate__(state)

    def forward(self, query, key, value, key_padding_mask=None,
                need_weights=True, attn_mask=None):
        # type: (Tensor, Tensor, Tensor, Optional[Tensor], bool, Optional[Tensor]) -> Tuple[Tensor, Optional[Tensor]]
        r"""
    Args:
        query, key, value: map a query and a set of key-value pairs to an output.
            See "Attention Is All You Need" for more details.
        key_padding_mask: if provided, specified padding elements in the key will
            be ignored by the attention. When given a binary mask and a value is True,
            the corresponding value on the attention layer will be ignored. When given
            a byte mask and a value is non-zero, the corresponding value on the attention
            layer will be ignored
        need_weights: output attn_output_weights.
        attn_mask: 2D or 3D mask that prevents attention to certain positions. A 2D mask will be broadcasted for all
            the batches while a 3D mask allows to specify a different mask for the entries of each batch.

    Shape:
        - Inputs:
        - query: :math:`(L, N, E)` where L is the target sequence length, N is the batch size, E is
          the embedding dimension.
        - key: :math:`(S, N, E)`, where S is the source sequence length, N is the batch size, E is
          the embedding dimension.
        - value: :math:`(S, N, E)` where S is the source sequence length, N is the batch size, E is
          the embedding dimension.
        - key_padding_mask: :math:`(N, S)` where N is the batch size, S is the source sequence length.
          If a ByteTensor is provided, the non-zero positions will be ignored while the position
          with the zero positions will be unchanged. If a BoolTensor is provided, the positions with the
          value of ``True`` will be ignored while the position with the value of ``False`` will be unchanged.
        - attn_mask: 2D mask :math:`(L, S)` where L is the target sequence length, S is the source sequence length.
          3D mask :math:`(N*num_heads, L, S)` where N is the batch size, L is the target sequence length,
          S is the source sequence length. attn_mask ensure that position i is allowed to attend the unmasked
          positions. If a ByteTensor is provided, the non-zero positions are not allowed to attend
          while the zero positions will be unchanged. If a BoolTensor is provided, positions with ``True``
          is not allowed to attend while ``False`` values will be unchanged. If a FloatTensor
          is provided, it will be added to the attention weight.

        - Outputs:
        - attn_output: :math:`(L, N, E)` where L is the target sequence length, N is the batch size,
          E is the embedding dimension.
        - attn_output_weights: :math:`(N, L, S)` where N is the batch size,
          L is the target sequence length, S is the source sequence length.
        """
        if not self._qkv_same_embed_dim:
            return multi_head_attention_forward2(
                query, key, value, self.embed_dim, self.num_heads,
                self.in_proj_weight, self.in_proj_bias,
                self.bias_k, self.bias_v, self.add_zero_attn,
                self.dropout, self.out_proj.weight, self.out_proj.bias,
                training=self.training,
                key_padding_mask=key_padding_mask, need_weights=need_weights,
                attn_mask=attn_mask, use_separate_proj_weight=True,
                q_proj_weight=self.q_proj_weight, k_proj_weight=self.k_proj_weight,
                v_proj_weight=self.v_proj_weight)
        else:
            return multi_head_attention_forward2(
                query, key, value, self.embed_dim, self.num_heads,
                self.in_proj_weight, self.in_proj_bias,
                self.bias_k, self.bias_v, self.add_zero_attn,
                self.dropout, self.out_proj.weight, self.out_proj.bias,
                training=self.training,
                key_padding_mask=key_padding_mask, need_weights=need_weights,
                attn_mask=attn_mask)

In [22]:
def dropout(input, p=0.5, training=True, inplace=False):
    # type: (Tensor, float, bool, bool) -> Tensor
    r"""
    During training, randomly zeroes some of the elements of the input
    tensor with probability :attr:`p` using samples from a Bernoulli
    distribution.
    See :class:`~torch.nn.Dropout` for details.
    Args:
        p: probability of an element to be zeroed. Default: 0.5
        training: apply dropout if is ``True``. Default: ``True``
        inplace: If set to ``True``, will do this operation in-place. Default: ``False``
    """
    assert(p == 0.0)
    return input
    if p < 0. or p > 1.:
        raise ValueError("dropout probability has to be between 0 and 1, "
                         "but got {}".format(p))
    return (_VF.dropout_(input, p, training)
            if inplace
            else _VF.dropout(input, p, training))


def softmax(input, dim=None, _stacklevel=3, dtype=None):
    # type: (Tensor, Optional[int], int, Optional[int]) -> Tensor
    r"""Applies a softmax function.
    Softmax is defined as:
    :math:`\text{Softmax}(x_{i}) = \frac{\exp(x_i)}{\sum_j \exp(x_j)}`
    It is applied to all slices along dim, and will re-scale them so that the elements
    lie in the range `[0, 1]` and sum to 1.
    See :class:`~torch.nn.Softmax` for more details.
    Arguments:
        input (Tensor): input
        dim (int): A dimension along which softmax will be computed.
        dtype (:class:`torch.dtype`, optional): the desired data type of returned tensor.
          If specified, the input tensor is casted to :attr:`dtype` before the operation
          is performed. This is useful for preventing data type overflows. Default: None.
    .. note::
        This function doesn't work directly with NLLLoss,
        which expects the Log to be computed between the Softmax and itself.
        Use log_softmax instead (it's faster and has better numerical properties).
    """
    if dim is None:
        dim = _get_softmax_dim('softmax', input.dim(), _stacklevel)
    if dtype is None:
        ret = input.softmax(dim)
    else:
        ret = input.softmax(dim, dtype=dtype)
    return ret

def linear(input, weight, bias=None):
    # type: (Tensor, Tensor, Optional[Tensor]) -> Tensor
    r"""
    Applies a linear transformation to the incoming data: :math:`y = xA^T + b`.
    This operator supports :ref:`TensorFloat32<tf32_on_ampere>`.
    Shape:
        - Input: :math:`(N, *, in\_features)` N is the batch size, `*` means any number of
          additional dimensions
        - Weight: :math:`(out\_features, in\_features)`
        - Bias: :math:`(out\_features)`
        - Output: :math:`(N, *, out\_features)`
    """
    tens_ops = (input, weight)
    if input.dim() == 2 and bias is not None:
        # fused op is marginally faster
        ret = torch.addmm(bias, input, weight.t())
    else:
        output = input.matmul(weight.t())
        if bias is not None:
            output += bias
        ret = output
    return ret

test = 0
def multi_head_attention_forward2(query: Tensor,
                                 key: Tensor,
                                 value: Tensor,
                                 embed_dim_to_check: int,
                                 num_heads: int,
                                 in_proj_weight: Tensor,
                                 in_proj_bias: Tensor,
                                 bias_k: Optional[Tensor],
                                 bias_v: Optional[Tensor],
                                 add_zero_attn: bool,
                                 dropout_p: float,
                                 out_proj_weight: Tensor,
                                 out_proj_bias: Tensor,
                                 training: bool = True,
                                 key_padding_mask: Optional[Tensor] = None,
                                 need_weights: bool = True,
                                 attn_mask: Optional[Tensor] = None,
                                 use_separate_proj_weight: bool = False,
                                 q_proj_weight: Optional[Tensor] = None,
                                 k_proj_weight: Optional[Tensor] = None,
                                 v_proj_weight: Optional[Tensor] = None,
                                 static_k: Optional[Tensor] = None,
                                 static_v: Optional[Tensor] = None
                                 ) -> Tuple[Tensor, Optional[Tensor]]:
    r"""
    Args:
        query, key, value: map a query and a set of key-value pairs to an output.
            See "Attention Is All You Need" for more details.
        embed_dim_to_check: total dimension of the model.
        num_heads: parallel attention heads.
        in_proj_weight, in_proj_bias: input projection weight and bias.
        bias_k, bias_v: bias of the key and value sequences to be added at dim=0.
        add_zero_attn: add a new batch of zeros to the key and
                       value sequences at dim=1.
        dropout_p: probability of an element to be zeroed.
        out_proj_weight, out_proj_bias: the output projection weight and bias.
        training: apply dropout if is ``True``.
        key_padding_mask: if provided, specified padding elements in the key will
            be ignored by the attention. This is an binary mask. When the value is True,
            the corresponding value on the attention layer will be filled with -inf.
        need_weights: output attn_output_weights.
        attn_mask: 2D or 3D mask that prevents attention to certain positions. A 2D mask will be broadcasted for all
            the batches while a 3D mask allows to specify a different mask for the entries of each batch.
        use_separate_proj_weight: the function accept the proj. weights for query, key,
            and value in different forms. If false, in_proj_weight will be used, which is
            a combination of q_proj_weight, k_proj_weight, v_proj_weight.
        q_proj_weight, k_proj_weight, v_proj_weight, in_proj_bias: input projection weight and bias.
        static_k, static_v: static key and value used for attention operators.
    Shape:
        Inputs:
        - query: :math:`(L, N, E)` where L is the target sequence length, N is the batch size, E is
          the embedding dimension.
        - key: :math:`(S, N, E)`, where S is the source sequence length, N is the batch size, E is
          the embedding dimension.
        - value: :math:`(S, N, E)` where S is the source sequence length, N is the batch size, E is
          the embedding dimension.
        - key_padding_mask: :math:`(N, S)` where N is the batch size, S is the source sequence length.
          If a ByteTensor is provided, the non-zero positions will be ignored while the zero positions
          will be unchanged. If a BoolTensor is provided, the positions with the
          value of ``True`` will be ignored while the position with the value of ``False`` will be unchanged.
        - attn_mask: 2D mask :math:`(L, S)` where L is the target sequence length, S is the source sequence length.
          3D mask :math:`(N*num_heads, L, S)` where N is the batch size, L is the target sequence length,
          S is the source sequence length. attn_mask ensures that position i is allowed to attend the unmasked
          positions. If a ByteTensor is provided, the non-zero positions are not allowed to attend
          while the zero positions will be unchanged. If a BoolTensor is provided, positions with ``True``
          are not allowed to attend while ``False`` values will be unchanged. If a FloatTensor
          is provided, it will be added to the attention weight.
        - static_k: :math:`(N*num_heads, S, E/num_heads)`, where S is the source sequence length,
          N is the batch size, E is the embedding dimension. E/num_heads is the head dimension.
        - static_v: :math:`(N*num_heads, S, E/num_heads)`, where S is the source sequence length,
          N is the batch size, E is the embedding dimension. E/num_heads is the head dimension.
        Outputs:
        - attn_output: :math:`(L, N, E)` where L is the target sequence length, N is the batch size,
          E is the embedding dimension.
        - attn_output_weights: :math:`(N, L, S)` where N is the batch size,
          L is the target sequence length, S is the source sequence length.
    """
    global test
    tgt_len, bsz, embed_dim = query.size()
    assert embed_dim == embed_dim_to_check
    # allow MHA to have different sizes for the feature dimension
    assert key.size(0) == value.size(0) and key.size(1) == value.size(1)

    head_dim = embed_dim // num_heads
    assert head_dim * num_heads == embed_dim, "embed_dim must be divisible by num_heads"
    scaling = float(head_dim) ** -0.5

    if not use_separate_proj_weight:
        if (query is key or torch.equal(query, key)) and (key is value or torch.equal(key, value)):
            # self-attention
            print('in_proj_weight', in_proj_weight)
            print('query', query)
            print(in_proj_bias)
            q, k, v = linear(query, in_proj_weight, in_proj_bias).chunk(3, dim=-1)
            print('q', q)
            print('here2')
            test = (in_proj_weight, in_proj_bias)
            print('here!')

        elif (key is value or torch.equal(key, value)):
            # encoder-decoder attention
            # This is inline in_proj function with in_proj_weight and in_proj_bias
            _b = in_proj_bias
            _start = 0
            _end = embed_dim
            _w = in_proj_weight[_start:_end, :]
            if _b is not None:
                _b = _b[_start:_end]
            q = linear(query, _w, _b)

            if key is None:
                assert value is None
                k = None
                v = None
            else:

                # This is inline in_proj function with in_proj_weight and in_proj_bias
                _b = in_proj_bias
                _start = embed_dim
                _end = None
                _w = in_proj_weight[_start:, :]
                if _b is not None:
                    _b = _b[_start:]
                k, v = linear(key, _w, _b).chunk(2, dim=-1)

        else:
            # This is inline in_proj function with in_proj_weight and in_proj_bias
            _b = in_proj_bias
            _start = 0
            _end = embed_dim
            _w = in_proj_weight[_start:_end, :]
            if _b is not None:
                _b = _b[_start:_end]
            q = linear(query, _w, _b)

            # This is inline in_proj function with in_proj_weight and in_proj_bias
            _b = in_proj_bias
            _start = embed_dim
            _end = embed_dim * 2
            _w = in_proj_weight[_start:_end, :]
            if _b is not None:
                _b = _b[_start:_end]
            k = linear(key, _w, _b)

            # This is inline in_proj function with in_proj_weight and in_proj_bias
            _b = in_proj_bias
            _start = embed_dim * 2
            _end = None
            _w = in_proj_weight[_start:, :]
            if _b is not None:
                _b = _b[_start:]
            v = linear(value, _w, _b)
    else:
        q_proj_weight_non_opt = torch.jit._unwrap_optional(q_proj_weight)
        len1, len2 = q_proj_weight_non_opt.size()
        assert len1 == embed_dim and len2 == query.size(-1)

        k_proj_weight_non_opt = torch.jit._unwrap_optional(k_proj_weight)
        len1, len2 = k_proj_weight_non_opt.size()
        assert len1 == embed_dim and len2 == key.size(-1)

        v_proj_weight_non_opt = torch.jit._unwrap_optional(v_proj_weight)
        len1, len2 = v_proj_weight_non_opt.size()
        assert len1 == embed_dim and len2 == value.size(-1)

        if in_proj_bias is not None:
            q = linear(query, q_proj_weight_non_opt, in_proj_bias[0:embed_dim])
            k = linear(key, k_proj_weight_non_opt, in_proj_bias[embed_dim:(embed_dim * 2)])
            v = linear(value, v_proj_weight_non_opt, in_proj_bias[(embed_dim * 2):])
        else:
            q = linear(query, q_proj_weight_non_opt, in_proj_bias)
            k = linear(key, k_proj_weight_non_opt, in_proj_bias)
            v = linear(value, v_proj_weight_non_opt, in_proj_bias)
    q = q * scaling
    
    print('q', q)
    print('k', k)
    print('v', v)

    if attn_mask is not None:
        assert attn_mask.dtype == torch.float32 or attn_mask.dtype == torch.float64 or \
            attn_mask.dtype == torch.float16 or attn_mask.dtype == torch.uint8 or attn_mask.dtype == torch.bool, \
            'Only float, byte, and bool types are supported for attn_mask, not {}'.format(attn_mask.dtype)
        if attn_mask.dtype == torch.uint8:
            warnings.warn("Byte tensor for attn_mask in nn.MultiheadAttention is deprecated. Use bool tensor instead.")
            attn_mask = attn_mask.to(torch.bool)

        if attn_mask.dim() == 2:
            attn_mask = attn_mask.unsqueeze(0)
            if list(attn_mask.size()) != [1, query.size(0), key.size(0)]:
                raise RuntimeError('The size of the 2D attn_mask is not correct.')
        elif attn_mask.dim() == 3:
            if list(attn_mask.size()) != [bsz * num_heads, query.size(0), key.size(0)]:
                raise RuntimeError('The size of the 3D attn_mask is not correct.')
        else:
            raise RuntimeError("attn_mask's dimension {} is not supported".format(attn_mask.dim()))
        # attn_mask's dim is 3 now.

    # convert ByteTensor key_padding_mask to bool
    if key_padding_mask is not None and key_padding_mask.dtype == torch.uint8:
        warnings.warn("Byte tensor for key_padding_mask in nn.MultiheadAttention is deprecated. Use bool tensor instead.")
        key_padding_mask = key_padding_mask.to(torch.bool)

    if bias_k is not None and bias_v is not None:
        if static_k is None and static_v is None:
            k = torch.cat([k, bias_k.repeat(1, bsz, 1)])
            v = torch.cat([v, bias_v.repeat(1, bsz, 1)])
            if attn_mask is not None:
                attn_mask = pad(attn_mask, (0, 1))
            if key_padding_mask is not None:
                key_padding_mask = pad(key_padding_mask, (0, 1))
        else:
            assert static_k is None, "bias cannot be added to static key."
            assert static_v is None, "bias cannot be added to static value."
    else:
        assert bias_k is None
        assert bias_v is None

    q = q.contiguous().view(tgt_len, bsz * num_heads, head_dim).transpose(0, 1)
    if k is not None:
        k = k.contiguous().view(-1, bsz * num_heads, head_dim).transpose(0, 1)
    if v is not None:
        v = v.contiguous().view(-1, bsz * num_heads, head_dim).transpose(0, 1)

    if static_k is not None:
        assert static_k.size(0) == bsz * num_heads
        assert static_k.size(2) == head_dim
        k = static_k

    if static_v is not None:
        assert static_v.size(0) == bsz * num_heads
        assert static_v.size(2) == head_dim
        v = static_v

    src_len = k.size(1)

    if key_padding_mask is not None:
        assert key_padding_mask.size(0) == bsz
        assert key_padding_mask.size(1) == src_len

    if add_zero_attn:
        src_len += 1
        k = torch.cat([k, torch.zeros((k.size(0), 1) + k.size()[2:], dtype=k.dtype, device=k.device)], dim=1)
        v = torch.cat([v, torch.zeros((v.size(0), 1) + v.size()[2:], dtype=v.dtype, device=v.device)], dim=1)
        if attn_mask is not None:
            attn_mask = pad(attn_mask, (0, 1))
        if key_padding_mask is not None:
            key_padding_mask = pad(key_padding_mask, (0, 1))

    attn_output_weights = torch.bmm(q, k.transpose(1, 2))
    print('presoftmax', attn_output_weights)
    assert list(attn_output_weights.size()) == [bsz * num_heads, tgt_len, src_len]

    if attn_mask is not None:
        if attn_mask.dtype == torch.bool:
            attn_output_weights.masked_fill_(attn_mask, float('-inf'))
        else:
            attn_output_weights += attn_mask


    if key_padding_mask is not None:
        attn_output_weights = attn_output_weights.view(bsz, num_heads, tgt_len, src_len)
        attn_output_weights = attn_output_weights.masked_fill(
            key_padding_mask.unsqueeze(1).unsqueeze(2),
            float('-inf'),
        )
        attn_output_weights = attn_output_weights.view(bsz * num_heads, tgt_len, src_len)

    attn_output_weights = softmax(
        attn_output_weights, dim=-1)
    print('post softmax', attn_output_weights)
    attn_output_weights = dropout(attn_output_weights, p=dropout_p, training=training)

    attn_output = torch.bmm(attn_output_weights, v)
    print(attn_output)
    assert list(attn_output.size()) == [bsz * num_heads, tgt_len, head_dim]
    attn_output = attn_output.transpose(0, 1).contiguous().view(tgt_len, bsz, embed_dim)
    attn_output = linear(attn_output, out_proj_weight, out_proj_bias)
    print('postlinear', attn_output)

    if need_weights:
        # average attention weights over heads
        attn_output_weights = attn_output_weights.view(bsz, num_heads, tgt_len, src_len)
        return attn_output, attn_output_weights.sum(dim=1) / num_heads
    else:
        return attn_output, None

In [23]:
batch_size = 1
embedding_size = 128
src_len = 5
model = MultiheadAttention2(embedding_size, 1, 0.0)
#saveStateDict(model, filepath)
src = torch.rand(src_len, batch_size, embedding_size)

In [24]:
output = model.forward(src, src, src)[0]

in_proj_weight Parameter containing:
tensor([[-0.0304,  0.0714, -0.0197,  ..., -0.0713,  0.0892, -0.0649],
        [ 0.0807, -0.0504, -0.0537,  ..., -0.0990,  0.0702, -0.1041],
        [-0.0684,  0.0077, -0.0212,  ..., -0.0112,  0.0275, -0.0939],
        ...,
        [-0.0832, -0.0930,  0.0146,  ..., -0.0329,  0.0885,  0.0669],
        [ 0.0751, -0.0531, -0.0746,  ..., -0.0936,  0.0032,  0.0827],
        [ 0.0973,  0.0723, -0.0582,  ..., -0.0217, -0.1075, -0.0729]],
       requires_grad=True)
query tensor([[[6.5580e-01, 6.7645e-01, 4.2053e-01, 8.9145e-01, 2.4312e-01,
          1.0258e-01, 7.4126e-01, 5.9265e-01, 7.3318e-01, 9.4323e-01,
          6.2272e-01, 9.7688e-01, 2.7367e-01, 6.0029e-01, 4.0472e-01,
          5.0296e-01, 2.3671e-01, 9.1568e-01, 3.0918e-01, 2.8747e-01,
          2.6097e-01, 1.7049e-01, 6.1015e-03, 1.9075e-02, 6.7696e-02,
          7.0685e-01, 5.5409e-01, 9.0990e-01, 6.4519e-01, 6.6409e-01,
          9.5939e-01, 6.8551e-01, 9.7827e-01, 6.7857e-01, 6.5455e-01,
      

In [25]:
filepath = '/private/home/padentomasello/scratch/pytorch_testing/multi_headed_attention.array'
af.array.save_array('input', toArrayFire(src), filepath, False)
i = 1
params = {}
for (name, param) in model.named_parameters():
        if 'in_proj' in name:
            q, k, v = param.chunk(3, dim=0)
            hack = '0'
            if 'in_proj_bias' in name: hack = '1'
            params['0q_' + hack + name] = q
            params['1k_' + hack + name] = k
            params['2v_' + hack + name] = v
            if 'in_proj_bias' in name:
                for key in sorted(params.keys()):
                    af_array = toArrayFire(params[key])
                    print(name)
                    if 'weight' in key:
                        print('here!')
                        af_array = af.array.transpose(af_array)
                    print(key, i)
                    af.array.save_array(key, af_array, filepath, True)  
                    i = i + 1
            continue
        elif len(param.size()) > 0:
            af_array = toArrayFire(param)
            if 'fc' in name and 'weight' in name:
                af_array = af.array.transpose(af_array)
            if 'weight' in name and 'proj' in name:
                af_array = af.array.transpose(af_array)
            print(name, i)
            af.array.save_array(name, af_array, filepath, True)
            i = i + 1
af.array.save_array('output', toArrayFire(output), filepath, True)

(128, 1, 5)
(128, 128)
in_proj_bias
here!
0q_0in_proj_weight 1
(128,)
in_proj_bias
0q_1in_proj_bias 2
(128, 128)
in_proj_bias
here!
1k_0in_proj_weight 3
(128,)
in_proj_bias
1k_1in_proj_bias 4
(128, 128)
in_proj_bias
here!
2v_0in_proj_weight 5
(128,)
in_proj_bias
2v_1in_proj_bias 6
(128, 128)
out_proj.weight 7
(128,)
out_proj.bias 8
(128, 1, 5)


9

In [26]:
output

tensor([[[-0.2186, -0.3842,  0.1497, -0.0263,  0.2757,  0.1261,  0.3305,
          -0.1523, -0.2219,  0.1608, -0.1299,  0.0411, -0.0869, -0.1442,
          -0.1239,  0.0169,  0.2050,  0.2134,  0.3478,  0.1704,  0.0294,
           0.3097,  0.1604,  0.0194, -0.1723, -0.4511, -0.1123, -0.3128,
           0.1281, -0.0255, -0.0620, -0.0702, -0.0028,  0.1525, -0.1405,
          -0.1809, -0.0257, -0.2073, -0.0193,  0.0517,  0.1479,  0.0442,
          -0.0223, -0.3561, -0.1487, -0.0483,  0.1664, -0.0244, -0.1497,
           0.1999,  0.2401,  0.0254, -0.0240,  0.2015,  0.3916, -0.3342,
           0.1651, -0.0227, -0.2236,  0.1835,  0.4124, -0.3942, -0.0274,
          -0.1710, -0.2303,  0.0193, -0.0628,  0.1685,  0.2286,  0.1842,
          -0.4431,  0.2175,  0.1231, -0.3336, -0.0163, -0.2438,  0.1562,
          -0.2061, -0.1103,  0.2375, -0.1800, -0.3421,  0.0936,  0.0660,
           0.0680,  0.2059,  0.1330,  0.1728, -0.0286, -0.3282,  0.2460,
          -0.0512, -0.6025,  0.0551, -0.1901, -0.05